<a href="https://colab.research.google.com/github/yk-Jeong/Kaggle-Studies/blob/main/N512_%EB%A6%AC%ED%8C%A9%ED%86%A0%EB%A7%81%EA%B3%BC_%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

타이타닉 데이터를 활용한 리팩토링 
<br>참고: https://deep-diver.github.io/pocket-ml-reference-korean/chapter3/

In [1]:
import pandas as pd

In [2]:
#data url 
url = "https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls"
df = pd.read_excel(url)
orig_df = df
df.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

전체 데이터 개형을 한번 살펴보는 것이 좋겠다는 생각이 들어서 

In [11]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
mask = df.isnull().any(axis=1) #.any는 무엇?
mask.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [4]:
df[mask].body.head()

0      NaN
1      NaN
2      NaN
3    135.0
4      NaN
Name: body, dtype: float64

In [6]:
#성비를 확인
df.sex.value_counts(dropna = False)

male      843
female    466
Name: sex, dtype: int64

In [7]:
df.embarked.value_counts(dropna = False)

S      914
C      270
Q      123
NaN      2
Name: embarked, dtype: int64

feature 생성 

In [8]:
name = df.name
name.head()

0                      Allen, Miss. Elisabeth Walton
1                     Allison, Master. Hudson Trevor
2                       Allison, Miss. Helen Loraine
3               Allison, Mr. Hudson Joshua Creighton
4    Allison, Mrs. Hudson J C (Bessie Waldo Daniels)
Name: name, dtype: object

In [15]:
#일부 피처를 drop
df = df.drop(
    columns = ["name", "ticket", "home.dest", "boat", "body", "cabin"]
)
df.columns

Index(['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked'],
      dtype='object')

In [17]:
y = df.survived
X = df.drop(columns='survived')
print(X, y)

      pclass     sex      age  sibsp  parch      fare embarked
0          1  female  29.0000      0      0  211.3375        S
1          1    male   0.9167      1      2  151.5500        S
2          1  female   2.0000      1      2  151.5500        S
3          1    male  30.0000      1      2  151.5500        S
4          1  female  25.0000      1      2  151.5500        S
...      ...     ...      ...    ...    ...       ...      ...
1304       3  female  14.5000      1      0   14.4542        C
1305       3  female      NaN      1      0   14.4542        C
1306       3    male  26.5000      0      0    7.2250        C
1307       3    male  27.0000      0      0    7.2250        C
1308       3    male  29.0000      0      0    7.8750        S

[1309 rows x 7 columns] 0       1
1       1
2       0
3       0
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: survived, Length: 1309, dtype: int64


sample data

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

없었던 부분

In [34]:
cols = "pclass,age,sibsp,fare".split(",")
sca = preprocessing.StandardScaler()

NameError: ignored

In [29]:
from sklearn.experimental import(
    enable_iterative_imputer
)
from sklearn import impute

리팩터링

In [20]:
def tweak_titanic(df):
  df = df.drop(
      columns = [
                 "name", "ticket", "home.dest", "boat", "body", "cabin"
      ]
  ).pipe(pd.get_dummies, drop_first = True)
  return df

In [31]:
def get_train_test_X_y(df, y_col, size=0.3, std_cols = None):
  y = df[y_col]
  X = df.drop(columns = y_col)
  X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size = size, random_state = 42
  )
  cols = X.columns
  num_cols = [
              "pclass", "age", "sibsp", "parch", "fare"
  ]
  fi = impute.IterativeImputer()

  X_train.loc[:, num_cols] = fi.fit_transform(X_train[num_cols])
  X_test.loc[:, num_cols] = fi.fit_transform(X_test[num_cols])

  if std_cols:
    std = preprocessing.StandardScaler()
    X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols])
    X_test.loc[:, std_cols] = std.fit_transform(X_test[std_cols])

  return X_train, X_test, y_train, y_test

In [32]:
ti_df = tweak_titanic(orig_df)
std_cols = "pclass, age, sibsp, fare".split(",")

In [36]:
X_train, X_test, y_train, y_test = get_train_test_X_y(ti_df, "survived", std_cols = std_cols)

NameError: ignored

모델 만들기

In [23]:
from sklearn import ensemble

In [35]:
rf = ensemble.RandomForestClassifier(n_estimators = 100, random_state = 42)
rf.fit(X_train, y_train)

ValueError: ignored

모델 평가 

In [ ]:
rf.score(X_test, y_tset)

In [ ]:
from 